In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Conv2D , Flatten , Dropout , MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import PIL.Image
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
data_dir = "/floyd/input/whaledetection"

In [3]:
from glob import glob

In [40]:
train_images = glob(data_dir + "/train/*jpg")
test_images = glob(data_dir+"/test/*jpg")
df = pd.read_csv(data_dir+"/train.csv")

df["Image"] = df["Image"].map( lambda x : data_dir + "/train/"+x)
ImageToLabelDict = dict( zip( df["Image"], df["Id"]))

In [5]:
SIZE = 64
#image are imported with a resizing and a black and white conversion
def ImportImage( filename):
    img = Image.open(filename).convert('RGB').resize( (SIZE,SIZE))
    #print(np.array(img).shape)
    return np.array(img)
train_img = np.array([ImportImage( img) for img in train_images])
x = train_img

In [6]:
print(x[0].shape)

(64, 64, 3)


In [7]:
#visualize the training distribution
# print( "Nbr of samples/class\tNbr of classes")
# for index, val in df["Id"].value_counts().value_counts().sort_index().iteritems():
#     print( "%d\t\t\t%d" %(index,val))

In [8]:
class LabelOneHotEncoder():
    def __init__(self):
        self.ohe = OneHotEncoder()
        self.le = LabelEncoder()
    def fit_transform(self, x):
        features = self.le.fit_transform( x)
        return self.ohe.fit_transform( features.reshape(-1,1))
    def transform( self, x):
        return self.ohe.transform( self.la.transform( x.reshape(-1,1)))
    def inverse_tranform( self, x):
        return self.le.inverse_transform( self.ohe.inverse_tranform( x))
    def inverse_labels( self, x):
        return self.le.inverse_transform( x)

y = list(map(ImageToLabelDict.get, train_images))
#y is the different category of the classes
lohe = LabelOneHotEncoder()
y_cat = lohe.fit_transform(y)

In [9]:
#reshaping the images (9850 x 64 x 64) == > (9850 x 64 x 64 x 3) for each color channel
x = x.reshape((-1,SIZE,SIZE,3))

In [12]:
# data augmentation in tensoflow
def convert(image, label):
  image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
  return image, label

def augment(image,label):
  image,label = convert(image, label)
  image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
  image = tf.image.resize_with_crop_or_pad(image, 34, 34) # Add 6 pixels of padding
  image = tf.image.random_crop(image, size=[28, 28, 1]) # Random crop back to 28x28
  image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness

  return image,label

In [10]:
x_train = x.astype("float32")
y_train = y_cat

In [11]:
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

#training the image preprocessing
image_gen.fit(x_train, augment=True)

In [12]:
batch_size = 128
#using imagenet dataset to 
train_generator  =image_gen.flow(x_train, y_train.toarray(), batch_size=batch_size)

In [13]:

from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = SIZE
WIDTH = SIZE

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 4s 0us/step


In [14]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model


In [15]:
from collections import Counter
classes = len(y_cat.toarray()[0])
classes

4251

In [16]:
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=classes)

In [20]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

NUM_EPOCHS = 15
BATCH_SIZE = 32
num_train_images = 9850

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [28]:
# filepath="/floyd/input/whaledetection/" + "ResNet50" + "_model_weights.h5"
# checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
# callbacks_list = [checkpoint]

# history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
#                                        steps_per_epoch=num_train_images // BATCH_SIZE, 
#                                        shuffle=True)


# plot_training(history)
finetune_model.save('/floyd/input/whaledetection/model.h5')

OSError: Unable to create file (unable to open file: name = '/floyd/input/whaledetection/model.h5', errno = 30, error message = 'Read-only file system', flags = 13, o_flags = 242)

In [ ]:
import warnings
from os.path import split

predicted=[]

for image in test_images:
    img = ImportImage( image)
    x = img.astype( "float32")
    #applying preprocessing to test images
    x = image_gen.standardize( x.reshape(-1,SIZE,SIZE,3))

    y = finetune_model.predict(x.reshape(1,SIZE,SIZE,3))
    predicted_args = np.argsort(y)[0][::-1][:5]
    predicted_tags = lohe.inverse_labels( predicted_args)
    image = split(image)[-1]
    predicted_tags = " ".join( predicted_tags)
    predicted.append([image, predicted_tags])

In [42]:
# test_images

In [49]:
WeightFunction = lambda x : 1./x**0.75
ClassLabel2Index = lambda x : lohe.le.inverse_tranform( [[x]])
CountDict = dict( df["Id"].value_counts())
class_weight_dic = { lohe.le.transform( [image_name])[0] : WeightFunction(count) for image_name, count in CountDict.items()}
del CountDict

In [50]:
import keras
batch_size = 128
num_classes = len(y_cat.toarray()[0])
epochs = 9
input_shape = x[0].shape

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

model = Sequential()
model.add(Conv2D(48, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(48, (3, 3), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (5, 5), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.33))
model.add(Flatten())
model.add(Dense(36, activation='sigmoid'))
model.add(Dropout(0.33))
model.add(Dense(36, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
model.summary()
model.fit_generator(image_gen.flow(x_train, y_train.toarray(), batch_size=batch_size),
          steps_per_epoch=  x_train.shape[0]//batch_size,
          epochs=epochs,
          verbose=1,
          class_weight=class_weight_dic)

x_train shape: (9850, 64, 64, 1)
9850 train samples
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 62, 62, 48)        480       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 60, 60, 48)        20784     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 30, 30, 48)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 26, 26, 48)        57648     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 48)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8, 8, 48)          0         
_________________________________________________________________
fl